In [2]:
# импорт библиотек
import pandas as pd
import os
import re

In [3]:
# чтение исходных таблиц (29 шт)
# выделение в отдельные столбцы названий федеральных округов, регионов, локализации опухолей и кода МКБ-10
file_list = []
path = '/Users/yana_shinkaryukmail.ru/Downloads/datasets/'
for fname in os.listdir(path):
    if 'xlsx' in fname and '057' not in fname and '054' not in fname:
        files = pd.read_excel(path + fname)
        files['loc'] = files.columns[0].split("\n")[2].split("(")[0]
        files['mkb_10'] = files.columns[0].split("\n")[2].split("(")[1].split(")")[0]
        files = files[3:]
        files.columns = ['region', 'completed', 'completed, %_of_new',\
               'not_completed', 'not_completed, %_of_new', 'surgical, %', \
               'radiation, %', 'medicinal, %', 'combined_exept_chem_rad, %', 'chemo_radiotherapy, %', 'loc', 'mkb_10']
        
        files['fo'] = files['region'].apply(lambda x: x if 'ФО' in x else None)
        files['fo'].fillna(method='ffill', inplace=True)
        files['region'] = files['region'].apply(lambda x: None if 'ФО' in x else x)
        files.dropna(axis=0, inplace=True)
        file_list.append(files)

In [4]:
# объединение считанных файлов в переменную data
data = pd.concat(file_list, axis=0)
data.head()

,region,completed,"completed, %_of_new",not_completed,"not_completed, %_of_new","surgical, %","radiation, %","medicinal, %","combined_exept_chem_rad, %","chemo_radiotherapy, %",loc,mkb_10,fo
4,Белгородская область,442,55.4,258,32.3,32.1,0,0,67.9,0,Молочная железа,С50,ЦЕНТРАЛЬНЫЙ ФО
5,Брянская область,358,58.3,178,29,28.5,0,0,71.5,0,Молочная железа,С50,ЦЕНТРАЛЬНЫЙ ФО
6,Владимирская область,418,59.2,166,23.5,12.9,0,0,87.1,0,Молочная железа,С50,ЦЕНТРАЛЬНЫЙ ФО
7,Воронежская область,563,50.3,358,32,20.6,0,0,79.4,0,Молочная железа,С50,ЦЕНТРАЛЬНЫЙ ФО
8,Ивановская область,297,56.9,120,23,24.2,0,0,75.8,0,Молочная железа,С50,ЦЕНТРАЛЬНЫЙ ФО


In [5]:
# выделение данных по детям в переменную childrens
data['is_child'] = data['loc'].apply(lambda x: 1 if 'детей' in x else 0)
childrens = data.loc[data['is_child'] == 1]
data = data.loc[data['is_child'] == 0]
data = data.drop(['is_child'], axis=1)

# удаление данных с суммами человек по регионам
data['loc'] = data['loc'].apply(lambda x: None if 'всего' in x else x)
data['mkb_10'] = data['mkb_10'].apply(lambda x: "С44" if 'кроме' in x else x)
data = data.query('mkb_10 != "С00-96"')

In [6]:
# выделение возрастных групп у детей
childrens = childrens.rename(columns={'loc': 'group'})
childrens['group'] = childrens['group'].apply(lambda x: "Дети 0-14 лет" if '14' in x else "Дети 0-17 лет")
childrens = childrens.drop(['is_child'], axis=1)

In [7]:
# ознакомление с таблицей 54 (суммы по всей рф)
rf = pd.read_excel('./2021_Таблица_054_Состояние_онко_помощи_в_РФ.xlsx')
rf = rf[6:]
rf.columns = ['loc', 'mkb_10', 'completed', 'completed, %_of_new',\
               'not_completed', 'not_completed, %_of_new', 'surgical, %', \
               'radiation, %', 'medicinal, %', 'combined_exept_chem_rad, %', 'chemo_radiotherapy, %']

rf.head()

,loc,mkb_10,completed,"completed, %_of_new",not_completed,"not_completed, %_of_new","surgical, %","radiation, %","medicinal, %","combined_exept_chem_rad, %","chemo_radiotherapy, %"
6,Губа,С00,1472,78.3,178,9.5,58.6,27.5,0,12.4,1.5
7,Полость рта,C01-09,3376,36.6,1635,17.7,38.2,12.3,0,36.9,12.6
8,Глотка,С10-13,1194,21.9,907,16.6,17.3,17.3,0,37.2,28.1
9,Пищевод,С15,1693,23.9,1403,19.8,41.9,1.5,0,43.7,12.9
10,Желудок,С16,8652,30,4572,15.9,57.8,0,0,42,0.2


In [8]:
# ознакомление с дополнительными сведениями таблицы 57
dop = pd.read_excel('./2021_Таблица_057_Состояние_онко_помощи_в_РФ.xlsx')

dop = dop[5:]
dop.columns = ['region','ref_abs', 'ref_%_of_nd', 'ref_I_II_st',\
               'ref_I_II_st, %_of_abs', 'contra_abs', 'contra_%_of_nd', \
               'contra_I_II_st', 'contra_I_II_st, %_of_abs', 'ambul', 'ambul, %_of_completed',\
               'completed_med', 'completed_med_hemobl', 'completed_rad', 'to_be_med', \
               'completed_med', 'completed_med, %', 'to_be_rad', \
               'completed_rad', 'completed_rad, %', 'to_be_comb', 'completed_comb', 'completed_comb, %'
              ]
# выделение в отдельные столбцы названий федеральных округов и регионов
dop['fo'] = dop['region'].apply(lambda x: x if 'ФО' in x else None)
dop[['fo']] = dop[['fo']].fillna(method='ffill')
dop['region'] = dop['region'].apply(lambda x: None if 'ФО' in x else x)
dop.dropna(axis=0, inplace=True)
dop.head()

,region,ref_abs,ref_%_of_nd,ref_I_II_st,"ref_I_II_st, %_of_abs",contra_abs,contra_%_of_nd,contra_I_II_st,"contra_I_II_st, %_of_abs",ambul,...,to_be_med,completed_med,"completed_med, %",to_be_rad,completed_rad,"completed_rad, %",to_be_comb,completed_comb,"completed_comb, %",fo
6,Белгородская область,90,1.3,30,33.3,132,1.9,22,16.7,713,...,4631,4560,98.5,2044,1991,97.4,2437,2394,98.2,ЦЕНТРАЛЬНЫЙ ФО
7,Брянская область,20,0.4,9,45,97,1.7,12,12.4,515,...,5716,5700,99.7,1438,1420,98.7,1587,1570,98.9,ЦЕНТРАЛЬНЫЙ ФО
8,Владимирская область,107,1.9,71,66.4,245,4.5,100,40.8,304,...,7443,6949,93.4,1347,1257,93.3,3187,2270,71.2,ЦЕНТРАЛЬНЫЙ ФО
9,Воронежская область,111,1.2,41,36.9,335,3.7,82,24.5,648,...,9896,9839,99.4,2653,2637,99.4,3912,3878,99.1,ЦЕНТРАЛЬНЫЙ ФО
10,Ивановская область,43,0.9,26,60.5,118,2.6,25,21.2,230,...,5103,5051,99,1478,1462,98.9,3373,3341,99.1,ЦЕНТРАЛЬНЫЙ ФО


In [9]:
# выделение в отдельные переменные данных по отказам, противопоказаниям к лечению, 
# амбулаторным пациентам и количеству подлежащих и получивших лечение пациентов
refuses = dop[['fo','region','ref_abs', 'ref_%_of_nd', 'ref_I_II_st', 'ref_I_II_st, %_of_abs']]
contra = dop[['fo','region','contra_abs', 'contra_%_of_nd', 'contra_I_II_st', 'contra_I_II_st, %_of_abs']]
amb = dop[['fo','region','ambul', 'ambul, %_of_completed']]
to_be_completed = dop[['fo','region','to_be_med', 'completed_med', 'completed_med_hemobl','completed_med, %',\
                   'to_be_rad', 'completed_rad', 'completed_rad, %', \
                   'to_be_comb', 'completed_comb', 'completed_comb, %'    
    ]]

In [10]:
# сохранение данных:
# взрослое население
data.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/data.csv', index= False)

In [11]:
# дети
childrens.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/childrens.csv', index= False)

In [12]:
# дополнительные сведения
dop.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/dop.csv', index= False)

In [13]:
# суммы по всей рф
rf.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/rf.csv', index= False)

In [14]:
# отказы от лечения
refuses.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/refuses.csv', index= False)

In [15]:
# противопоказания к лечению
contra.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/contra.csv', index= False)

In [16]:
# амбулаторные пациенты
amb.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/amb.csv', index= False)

In [17]:
# подлежащие и получившие лечение
to_be_completed.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/datasets/to_be_completed.csv', index= False)